In [16]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
import numpy as np
import pandas as pd
import psycopg2
from dotenv import load_dotenv

from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [5]:
# Загрузка переменных окружения
load_dotenv()

table_name = 'time_series_cases'

In [6]:
# Подключаемся к базе
conn = psycopg2.connect()

In [7]:
# Загружаем в DataFrame напрямую
df = pd.read_sql(f"SELECT * FROM {table_name};", conn)

C:\Users\uzer\AppData\Local\Temp\ipykernel_14848\3455865493.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table_name};", conn)


In [12]:
conn.close()

In [10]:
# Вывод содержимого таблицы
df[[col for col in df.columns if col not in ['id']]].head(3)

,description,embedding,n_obs,periodicity_sec,regular_fraction,transition,shifting,seasonality,trend,stationarity,correlation,name
0,METR-LA - датасет дорожного трафика Лос-Анджел...,"[-0.017407065,0.026607541,-0.0031739117,0.0480...",34272,300,True,0.004425,0.030084,0.521438,0.269346,2.477447e-17,0.778835,METR-LA
1,PEMS-BAY — датасет дорожного трафика района за...,"[0.005604299,0.041165046,-0.02515411,0.0196176...",52116,300,True,0.005024,0.090430,0.629332,0.144609,7.554316e-28,0.842455,PEMS-BAY
2,PEMS04 — датасет дорожного трафика округа Лос-...,"[0.011207809,0.059337948,-0.039728384,0.035950...",16992,300,True,0.006927,0.084752,0.917431,0.338526,7.156624e-26,0.796511,PEMS04


In [14]:
# Вывод одного элемента
first_row = df.iloc[0].to_dict()

for k, v in first_row.items():
    if k != 'id':
        print(f"{k}: {v}")

description: METR-LA - датасет дорожного трафика Лос-Анджелеса. Содержит измерения средней скорости движения транспорта (миль/час) по дорожным сенсорам. Каждый канал - один сенсор. Данные собраны с марта по июнь 2012 года. Целевая переменная — средняя скорость движения по каждому сенсору. Экзогенные переменные отсутствуют. Ряды вещественные, категориальные данные отсутствуют. Наблюдается сезонность.
embedding: [-0.017407065,0.026607541,-0.0031739117,0.048056845,0.04181841,-0.014977716,0.04907943,0.0043289945,-0.012887002,-0.0016262869,-0.0010389636,-0.04493964,-0.026418844,-0.025298353,0.030192206,0.05681041,0.028579602,-0.005627207,-0.015731525,-0.031711243,-0.034879796,0.018233655,0.052463643,0.006911797,0.008697733,0.020099906,-0.026465405,0.015298474,0.016936561,0.011741715,0.04080274,0.019596828,-0.009540822,-0.021885706,-0.00093960064,-0.0057094996,-0.04101907,0.03178786,-0.024270568,0.01049869,0.008552889,0.03226594,-0.006080857,0.031022295,-0.018623907,-0.011073489,0.013977163,

In [15]:
first_row['description']

'METR-LA - датасет дорожного трафика Лос-Анджелеса. Содержит измерения средней скорости движения транспорта (миль/час) по дорожным сенсорам. Каждый канал - один сенсор. Данные собраны с марта по июнь 2012 года. Целевая переменная — средняя скорость движения по каждому сенсору. Экзогенные переменные отсутствуют. Ряды вещественные, категориальные данные отсутствуют. Наблюдается сезонность.'

In [19]:
# Новая задача
new_case = {
    'name': 'logistic_taask',
    'description': 'Данные по запасу. Каждый канал соответствует одному материалу, одно значение - это запас на складе в определенный день' + \
                    'Экзогенные переменные отсутствуют.'
}

In [21]:
# Получаем векторное представление описания задачи
model = SentenceTransformer('ai-forever/ru-en-RoSBERTa')
new_case_emb = model.encode(new_case['description'], convert_to_numpy=True, normalize_embeddings=True)

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
new_case_emb

array([-0.00353864,  0.041132  , -0.02103208, ...,  0.02121442,
       -0.01450895, -0.03039828], dtype=float32)

In [44]:
# df["embedding"] = df["embedding"].apply(lambda x: np.array(literal_eval(x), dtype=float))

# Матрица всех эмбеддингов
embeddings_matrix = np.vstack(df["embedding"])

# Косинусное сходство
sims = cosine_similarity(new_case_emb.reshape(1, -1), embeddings_matrix)[0]

In [45]:
sims

array([0.63035776, 0.60028607, 0.52623515, 0.53111667, 0.57704144,
       0.67060742, 0.61611539, 0.56650864, 0.47662649])

In [50]:
df['similarity'] = sims

# Выводим 3 наиболее похожих задач
for index in df.nlargest(3, 'similarity').index:
    print(f"name: {df.loc[index, 'name']}")
    print(f"description: {df.loc[index, 'description']}")
    print(f"similarity: {df.loc[index, 'similarity']:.4f}\n")

name: Electricity
description: Electricity — датасет почасового потребления электроэнергии по отдельным потребителям в Австралии (один канал относится к одному потребителю). Содержит измерения объёма потреблённой электроэнергии (в кВт·ч) для каждого потребителя. Данные собраны за период с 2012 по 2014 год с шагом 1 час. Целевая переменная — объём потребления электроэнергии (кВт·ч) для каждого потребителя. Экзогенные переменные отсутствуют. Ряды вещесвтенные, регулярные. Категориальные факторы отсутствуют.
similarity: 0.6706

name: METR-LA
description: METR-LA - датасет дорожного трафика Лос-Анджелеса. Содержит измерения средней скорости движения транспорта (миль/час) по дорожным сенсорам. Каждый канал - один сенсор. Данные собраны с марта по июнь 2012 года. Целевая переменная — средняя скорость движения по каждому сенсору. Экзогенные переменные отсутствуют. Ряды вещественные, категориальные данные отсутствуют. Наблюдается сезонность.
similarity: 0.6304

name: Solar
description: Solar —